# An End-to-End Trainable Neural Network for Image-based Sequence Recognition and Its Application to Scene Text Recognition  

## Abstract  

基于图像的序列识别一直是计算机视觉中长期存在的研究课题。在本文中，我们研究了场景文本识别的问题，这是基于图像的序列识别中最重要和最具挑战性的任务之一。提出了一种将特征提取，序列建模和 transcription 整合到统一框架中的新型神经网络架构。与之前的场景文本识别系统相比，所提出的架构具有四个不同的特性：（1）与大多数现有的组件需要单独训练和协调的算法相比，它是端对端训练的。（2）它自然地处理任意长度的序列，不涉及字符分割或水平尺度标准化。（3）它不仅限于任何预定义的词汇，并且在无词典和基于词典的场景文本识别任务中都取得了显著的表现。（4）它产生了一个有效而小得多的模型，这对于现实世界的应用场景更为实用。在包括 IIIT-5K，Street View Text 和 ICDAR 数据集在内的标准基准数据集上的实验证明了提出的算法比现有的技术更有优势。此外，提出的算法在基于图像的音乐乐谱识别任务中表现良好，这显然证实了模型的泛化能力。  

## 1. Introduction  

最近，社区已经看到神经网络的强大复兴，这主要受到深度神经网络模型，特别是深度卷积神经网络（DCNN）在各种视觉任务中的巨大成功的推动。然而，最近大多数与深度神经网络相关的工作主要致力于检测或对象分类。在本文中，我们关注计算机视觉中的一个经典问题：基于图像的序列识别。在现实世界中，稳定的视觉对象，如场景文字，手写字符和乐谱，往往以序列的形式出现，而不是孤立地出现。与一般的对象识别不同，识别这样的序列化对象通常需要系统预测一系列对象标签，而不是单个标签。因此，可以自然地将这样的对象的识别作为序列识别问题。序列化对象的另一个独特之处在于它们的长度可能会有很大变化。例如，英文单词可以由 2 个字符组成，如“OK”，或由 15 个字符组成，如“congratulations”。因此，最流行的深度模型像 DCNN 不能直接应用于序列预测，因为 DCNN 模型通常对具有固定维度的输入和输出进行操作，因此不能产生可变长度的标签序列。  

已经针对特定的序列化对象（例如场景文本）进行了一些尝试来解决该问题。例如，[35,8]中的算法首先检测单个字符，然后用 DCNN 模型识别这些检测到的字符，并使用标注的字符图像进行训练。这些方法通常需要训练强字符检测器，以便从原始单词图像中准确地检测和裁剪每个字符。一些其他方法（如[22]）将场景文本识别视为图像分类问题，并为每个英文单词（总共 9 万个词）分配一个标签，结果是一个大的训练模型中有很多类，这很难泛化到其它类型的序列化对象，如中文文本，音乐配乐等，因为这种序列的基本组合数目可能大于 100 万。总之，目前基于 DCNN 的系统不能直接用于基于图像的序列识别。  

循环神经网络（RNN）模型是深度神经网络家族中的另一个重要分支，主要设计来处理序列数据。RNN 的优点之一是在训练和测试中不需要序列目标图像中每个元素的位置。然而，将输入目标图像转换成图像特征序列的预处理步骤通常是必需的。例如，Graves 等人从手写文本中提取一系列几何或图像特征，而 Su 和 Lu 将字符图像转换为序列 HOG 特征。预处理步骤独立于流程中的后续组件，因此基于 RNN 的现有系统不能以端到端的方式进行训练和优化。 

一些不是基于神经网络的传统场景文本识别方法也为这一领域带来了 insightful 的想法和新颖的效果。例如，Almazan 等人和 Rodriguez-Serrano 等人提出将单词图像和文本字符串嵌入到公共向量子空间中，并将词识别转换为检索问题。Yao 等人和 Gordo 等人使用中层特征进行场景文本识别。虽然在标准基准数据集上取得了有效的性能，但是前面的基于神经网络的算法以及本文提出的方法通常都优于这些方法。  

本文的主要贡献了一种新颖的神经网络模型，其网络架构专门用于识别图像中的序列化对象。所提出的神经网络模型被称为卷积循环神经网络（CRNN），因为它是 DCNN 和 RNN 的组合。对于序列化对象，CRNN 与传统神经网络模型相比具有一些独特的优点：1）可以直接从序列标签（例如单词）学习，不需要详细的标注（例如字符）；2）直接从图像数据学习信息表示时具有与 DCNN 相同的性质，既不需要手工特征也不需要预处理步骤，包括二值化/分割，组件定位等；3）具有与 RNN 相同的性质，能够产生一系列标签；4）对类序列对象的长度无约束，只需要在训练阶段和测试阶段对高度进行归一化；5）与现有技术相比，它在场景文本（字识别）上获得更好或更具竞争力的表现。6）它比标准 DCNN 模型包含的参数要少得多，占用更少的存储空间。  

## 2. The Proposed Network Architecture  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/1.jpg?raw=true)  

如图 1 所示，CRNN 的网络架构由三部分组成，包括卷积层，循环层和 transcription 层，从底向上。  

在 CRNN 的底部，卷积层自动从每个输入图像中提取特征序列。在卷积网络之上，构建了一个循环网络，用于对卷积层输出的特征序列的每一帧进行预测。CRNN 顶部的 transcription 层将循环层的每帧预测转化为标签序列。虽然 CRNN 由不同类型的网络架构（如 CNN 和 RNN）组成，但可以通过一个损失函数进行联合训练。   

### 2.1. Feature Sequence Extraction  

在 CRNN 模型中，通过采用标准 CNN 模型（去除全连接层）中的卷积层和最大池化层来构造卷积层的组件。这样的组件用于从输入图像中提取序列特征表示。在进入网络之前，所有的图像需要缩放到相同的高度。然后从卷积层组件产生的特征图中提取特征向量序列，这些特征向量序列作为循环层的输入。具体地，特征序列的每一个特征向量在特征图上按列从左到右生成。这意味着第 i 个特征向量是所有特征图第 i 列的拼接。在我们的设置中每列的宽度固定为单个像素。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/2.jpg?raw=true)  

由于卷积层，最大池化层和元素激活函数在局部区域上执行，因此它们是平移不变的。因此，特征图的每列对应于原始图像的一个矩形区域，并且这些矩形区域与特征图上从左到右的相应列具有相同的顺序。如图 2 所示，特征序列中的每个向量关联一个感受野，并且可以被认为是该区域的图像描述符。  

鲁棒的，丰富的和可训练的深度卷积特征已被广泛应用于各种视觉识别任务。一些以前的方法已经使用 CNN 来学习诸如场景文本之类的序列化对象的鲁棒表示。然而，这些方法通常通过 CNN 提取整个图像的整体表示，然后收集局部深度特征来识别序列化对象的每个分量。由于 CNN 要求将输入图像缩放到固定尺寸，以满足其固定的输入尺寸，因为它们的长度变化很大，因此不适合序列化对象。在 CRNN 中，我们将深度特征传递到序列表示中，以便对类序列对象的长度变化保持不变。  

### 2.2. Sequence Labeling  

一个深度双向循环神经网络是建立在卷积层的顶部，作为循环层。循环层预测特征序列 $x=x_1,...,x_T$ 中每一帧 $x_t$ 的标签分布 $y_t$。循环层的优点有三个。首先，RNN 具有很强的捕获序列内上下文信息的能力。对于基于图像的序列识别使用上下文提示比独立处理每个符号更稳定且更有帮助。以场景文本识别为例，宽字符可能需要一些连续的帧来完全描述（参见图 2）。此外，一些模糊的字符在观察其上下文时更容易区分，例如，通过对比字符高度更容易识别“il”而不是分别识别它们中的每一个。其次，RNN 可以将误差差值反向传播到其输入，即卷积层，从而允许我们在统一的网络中共同训练循环层和卷积层。第三，RNN 能够从头到尾对任意长度的序列进行操作。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/3.jpg?raw=true)  

传统的 RNN 单元在其输入和输出层之间具有自连接的隐藏层。每次接收到序列中的帧 $x_t$ 时，它将使用非线性函数来更新其内部状态 $h_t$，该非线性函数同时接收当前输入 $x_t$ 和过去状态 $h_{t−1}$ 作为其输入：$h_t = g(x_t,h_{t-1})$。那么预测 $y_t$ 是基于 $h_t$ 的。以这种方式，过去的上下文 $\{x_{t'}\}_{t'< t}$ 被捕获并用于预测。然而，传统的 RNN 单元有梯度消失的问题，这限制了其可以存储的上下文范围，并给训练过程增加了负担。长短时记忆（LSTM）是一种专门设计用于解决这个问题的 RNN 单元。LSTM（图 3 所示）由一个存储单元和三个多重门组成，即输入，输出和遗忘门。在概念上，存储单元存储过去的上下文，并且输入和输出门允许单元长时间地存储上下文。同时，单元中的存储可以被遗忘门清除。LSTM 的特殊设计允许它捕获长距离依赖，这经常发生在基于图像的序列中。  

LSTM 是定向的，它只使用过去的上下文。然而，在基于图像的序列中，两个方向的上下文是相互有用且互补的。因此，我们遵循[17]，将两个 LSTM，一个向前和一个向后组合到一个双向 LSTM 中。此外，可以堆叠多个双向 LSTM，得到如图 3.b 所示的深双向 LSTM。深层结构允许比浅层抽象更高层次的抽象，并且在语音识别任务中取得了显著的性能改进。  

在循环层中，误差在图 3.b 所示箭头的相反方向传播，即反向传播时间（BPTT）。在循环层的底部，传播差异的序列被连接成映射，将特征映射转换为特征序列的操作进行反转并反馈到卷积层。实际上，我们创建一个称为“Map-to-Sequence”的自定义网络层，作为卷积层和循环层之间的桥梁。  

### 2.3. Transcription

转录是将 RNN 所做的每帧预测转换成标签序列的过程。数学上，转录是根据每帧预测找到具有最高概率的标签序列。在实践中，存在两种转录模式，即无词典转录和基于词典的转录。词典是一组标签序列，预测受拼写检查字典约束。在无词典模式中，预测时没有任何词典。在基于词典的模式中，通过选择具有最高概率的标签序列进行预测。  

#### 2.3.1 Probability of label sequence  

我们采用 Graves 等人提出的 Connectionist Temporal Classification（CTC）层中定义的条件概率。按照每帧预测结果 $y = y_1,...,y_T$ 对计算标签序列 $l$ 的概率，并忽略 $l$ 中每个标签所在的位置。因此，当我们使用这种概率的负对数似然作为训练网络的目标函数时，我们只需要图像及其相应的标签序列，避免了标注单个字符位置的劳动。  

条件概率的公式简要描述如下：输入是序列 $y = y_1,...,y_T$，其中 T 是序列长度。这里，每个 $y_t \in R^{\mid L' \mid}$ 是在集合 $L' = L \bigcup$ 上的概率分布，其中 L 包含了任务中的所有标签（例如，所有英文字符），以及由 - 表示的“空白”标签。序列到序列的映射函数 BB 定义在序列 $\pi \in L'^T$ 上，其中 T 是长度。BB 通过首先删除重复的标签，然后删除 blank，将 $\pi$ 映射到 $l$ 上。例如，BB 将“–hh-e-l-ll-oo–”（- 表示blank）映射到“hello”。然后，条件概率被定义为由 BB 将所有 $\pi$ 映射到 $l$ 上的概率之和：  

$p(l|y) = \sum_{\pi:B(\pi)=1} p(\pi|y) \tag{1}$

其中，$\pi$ 的概率定义为 $p(\pi|y) = \quad_{t=1}{T} y_{\pi_t}^t, \ y_{\pi_t}^t$ 是时刻 t 时有标签 $\pi_t$ 的概率。由于存在指数级数量的求和项，直接计算上述方程在计算上是不可行的。然而，使用[15]中描述的前向算法可以有效的进行计算。  

#### 2.3.2 Lexicon-free transcription  

在这种模式下，将具有方程 1 中定义的最高概率的序列 $l^*$ 作为预测。由于不存在用于精确找到解的可行方法，我们采用[15]中的策略。序列 $l^*$ 通过 $l^* \approx B(arg max_{\pi} p(\pi|y))$ 近似发现，e.g 在每个时间戳 t 采用最大概率的标签 $\pi_t$，并将结果序列映射到 $l^*$。  

#### 2.3.3 Lexicon-based transcription  

在基于字典的模式中，每个测试样本与词典 D 相关联。基本上，通过选择词典中具有方程 1 中定义的最高条件概率的序列来识别标签序列。然而，对于大型词典，例如 5 万个词的 Hunspell 拼写检查词典，对词典进行详尽的搜索是非常耗时的，即对词典中的所有序列计算方程 1，并选择概率最高的一个。为了解决这个问题，我们观察到，2.3.2 中描述的通过无词典转录预测的标签序列通常在编辑距离度量下接近于实际结果。这表示我们可以将搜索限制在最近邻候选目标 $N_{\delta}(l')$，其中 $\delta$ 是最大编辑距离，$l'$ 是在无词典模式下从 y 转录的序列：  

$l^* = arg max_{l \in N_{\delta}(l')} p(l|y) \tag{2}$

可以使用 BK 树数据结构有效地找到候选目标 $N_{\delta}(l')$，这是一种专门适用于离散度量空间的度量树。BK 树的搜索时间复杂度为 $O(log|D|)$，其中 |D| 是词典大小。因此，这个方案很容易扩展到非常大的词典。在我们的方法中，一个词典离线构造一个 BK 树。然后，我们使用树执行快速在线搜索，通过查找具有小于或等于 $\delta$ 编辑距离来查询序列。

### 2.4. Network Training  

用 $X = \{I_i,l_j\}_i$ 表示训练数据集，其中 $I_i$ 是训练图像，$l_i$ 是真实标签序列。目标是最小化真实条件概率的负对数似然：  

$O = - \sum_{I_i,l_i \in X} \log p(l_i|y_i) \tag{3}$

$y_i$ 是循环层和卷积层从 $I_i$ 生成的序列。目标函数直接从图像和它的真实标签序列计算代价值。因此，网络可以在成对的图像和序列上进行端对端训练，去除了在训练图像中手动标记所有单独组件的过程。  

用随机梯度下降（SGD）训练网络。梯度由反向传播算法计算。特别是，在转录层中，错误差异使用前向后向算法反向传播，如[15]中所述。在循环层中，应用反向传播时间（BPTT）来计算误差差异。  

为了优化，我们使用 ADADELTA 自动计算每维度学习率。与传统动量方法相比，ADADELTA 不需要手动设置学习速率。更重要的是，我们发现使用 ADADELTA的优化比动量方法更快收敛。  

## 3. Experiments  

为了评估提出的 CRNN 模型的有效性，我们在场景文本识别和乐谱识别的标准基准数据集上进行了实验，这些都是具有挑战性的视觉任务。数据集和训练测试的设置见 3.1 小节，场景文本图像中 CRNN 的详细设置见 3.2 小节，综合比较的结果在 3.3 小节报告。为了进一步证明 CRNN 的泛化性，在 3.4 小节我们在乐谱识别任务上验证了提出的算法。  

### 3.1. Datasets  

对于场景文本识别的所有实验，我们使用 Jaderberg 等人发布的合成数据集（Synth）作为训练数据。数据集包含 8 百万训练图像及其对应的实际单词。这样的图像由合成文本引擎生成并且是非常现实的。我们的网络在合成数据上进行了一次训练，并在所有其它现实世界的测试数据集上进行了测试，而没有在其训练数据上进行任何微调。即使 CRNN 模型是在纯合成文本数据上训练，但它在标准文本识别基准数据集的真实图像上工作良好。  

有四个流行的基准数据集用于场景文本识别的性能评估，即 ICDAR 2003（IC03），ICDAR 2013（IC13），IIIT 5k-word（IIIT5k）和 Street View Text (SVT)。  

IC03 测试数据集包含 251 个具有标记文本边界框的场景图像。我们忽略包含非字母数字字符或少于三个字符的图像，并获得具有 860 个裁剪的文本图像的测试集。每张测试图像与由 Wang 等人定义的 50 词的词典相关联。通过组合所有的每张图像词汇构建完整的词典。此外，我们使用由 Hunspell 拼写检查字典中的单词组成的 5 万个词的词典。  

IC13 测试数据集继承了 IC03 中的大部分数据。它包含 1015 个实际的裁剪单词图像。  

IIIT5k 包含从互联网收集的 3000 张裁剪的词测试图像。每张图像关联一个 50 词的词典和一个 1000 词的词典。  

SVT 测试数据集由从 Google 街景视图收集的 249 张街景图像组成。从它们中裁剪出了 647 张词图像。每张单词图像都有一个由 Wang 等人定义的 50 个词的词典。  

### 3.2. Implementation Details  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/4.jpg?raw=true)  

在实验中我们使用的网络配置总结在表 1 中。卷积层的架构是基于 VGG-VeryDeep 的架构。为了使其适用于识别英文文本，对其进行了调整。在第 3 和第4 个最大池化层中，我们采用 1×2 大小的矩形池化窗口而不是传统的平方形。这种调整产生宽度较大的特征图，因此具有更长的特征序列。例如，包含 10 个字符的图像通常为大小为 100×32，可以从其生成 25 帧的特征序列。这个长度超过了大多数英文单词的长度。最重要的是，矩形池窗口产生矩形感受野（如图 2 所示），这有助于识别一些具有窄形状的字符，例如 i 和 l。  

不仅有卷积层，而且还有循环层。众所周知两者都难以训练。我们发现批归一化技术对于训练这种深度网络非常有用。分别在第 5 和第 6 卷积层之后插入两个批归一化层。使用批归一化层训练过程大大加快。  

我们在 Torch7 框架内实现了网络，使用定制实现的 LSTM 单元（Torch7/CUDA），转录层（C++）和 BK 树数据结构（C++）。实验在具有 2.50 GHz Intel（R）Xeon E5-2609 CPU，64GB RAM 和 NVIDIA（R）Tesla(TM) K40 GPU 的工作站上进行。网络用 ADADELTA 训练，将参数ρ设置为 0.9。在训练期间，所有图像都被缩放为 100×32，以加快训练过程。训练过程大约需要 50 个小时才能达到收敛。测试图像缩放的高度为 32。宽度与高度成比例地缩放，但至少为 100 像素。平均测试时间为 0.16s/样本，在 IC03 上测得的，没有词典。近似词典搜索应用于 IC03 的 50k 词典，参数 $delta$ 设置为3。测试每个样本平均花费 0.53s。    

### 3.3. Comparative Evaluation  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/6.jpg?raw=true)  

提出的 CRNN 模型在上述四个公共数据集上获得的所有识别精度以及最近的包括基于深度模型的最新技术的识别精度，如表 2 所示。  

在有约束词典的情况中，我们的方法始终优于大多数最新的方法，并且平均打败了[22]中提出的最佳文本阅读器。具体来说，与[22]相比，我们在 IIIT5k 和 SVT 上获得了卓越的性能，仅在 IC03 上通过“Full”词典实现了较低性能。请注意，[22]中的模型是在特定字典上训练的，即每个单词都与一个类标签相关联。与[22]不同，CRNN 不限于识别已知字典中的单词，并且能够处理随机字符串（例如电话号码），句子或其他诸如中文单词的脚本。因此，CRNN 的结果在所有测试数据集上都具有竞争力。  

在无约束词典的情况下，我们的方法在 SVT 上仍取得了最佳性能，但在 IC03 和 IC13 上仍然落后于一些方法。注意，表 2 的“none”列中的空白表示这种方法不能应用于没有词典的识别，或者在无约束的情况下不能报告识别精度。我们的方法只使用具有单词级标签的合成文本作为训练数据，与PhotoOCR 非常不同，后者使用 790 万个具有字符级标注的真实单词图像进行训练。[22]中报告的最佳性能是在无约束词典的情况下，受益于它的大字典，然而，它不是前面提到的严格的无约束词典模型。在这个意义上，我们在无限制词典表中的结果仍然是有前途的。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/5.jpg?raw=true)  

为了进一步了解与其它文本识别方法相比，所提出算法的优点，我们提供了在一些特性上的综合比较，这些特性名称为 E2E Train，Conv Ftrs，CharGT-Free，Unconstrained 和 Model Size，如表 3 所示。  

E2E Train：这一列是为了显示某种文字阅读模型是否可以进行端到端的训练，无需任何预处理或经过几个分离的步骤，这表明这种方法对于训练是优雅且干净的。从表 3 可以看出，只有基于深度神经网络的模型，包括[22,21]以及CRNN具有这种性质。  

Conv Ftrs：这一列用来表明一个方法是否使用从训练图像直接学习到的卷积特征或手动特征作为基本的表示。  

CharGT-Free：这一列用来表明字符级标注对于训练模型是否是必要的。由于 CRNN 的输入和输出标签是序列，因此字符级标注是不必要的。  

Unconstrained：这一列用来表明训练模型是否受限于一个特定的字典，是否不能处理字典之外的单词或随机序列。注意尽管最近通过标签嵌入[5, 14]和增强学习学习到的模型取得了非常有竞争力的性能，但它们受限于一个特定的字典。  

Model Size：这一列报告了学习模型的存储空间。在 CRNN 中，所有的层有权重共享连接，不需要全连接层。因此，CRNN 的参数数量远小于 CNN 变体所得到的模型，与[22,21]相比，模型要小得多。我们的模型有 830 万个参数，只有 33MB RAM（每个参数使用 4 字节单精度浮点数），因此可以轻松地移植到移动设备上。  
  
表 3 详细列出了不同方法之间的差异，充分展示了 CRNN 与其它竞争方法的优势。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/7.jpg?raw=true)  

另外，为了测试参数 $\delta$ 的影响，我们在方程 2 中实验了 $\delta$ 的不同值。在图 4 中，我们将识别精度绘制为 $\delta$ 的函数。更大的 $\delta$ 导致更多的候选目标，从而基于词典的转录更准确。另一方面，由于更长的 BK 树搜索时间，以及更大数量的候选序列用于测试，计算成本随着 $\delta$ 的增大而增加。实际上，我们选择 $\delta = 3$ 作为精度和速度之间的折衷。  

### 3.4. Musical Score Recognition  

乐谱通常由排列在五线谱的音符序列组成。识别图像中的乐谱被称为光学音乐识别（OMR）问题。以前的方法通常需要图像预处理（主要是二值化），五线谱检测和单个音符识别。我们将 OMR 作为序列识别问题，直接用 CRNN 从图像中预测音符的序列。为了简单起见，我们仅识别音调，忽略所有和弦，并假定所有乐谱具有相同的大调音阶（C 大调）。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/8.jpg?raw=true)  

据我们所知，没有用于评估音调识别算法的公共数据集。为了准备 CRNN 所需的训练数据，我们从[2]中收集了 2650 张图像。每个图像中有一个包含 3 到20 个音符的乐谱片段。我们手动标记所有图像的真实标签序列（不是的音调序列）。收集到的图像通过旋转，缩放和用噪声损坏增强到了 265k 个训练样本，并用自然图像替换它们的背景。对于测试，我们创建了三个数据集：1）“纯净的”，其中包含从[2]收集的 260 张图像。实例如图 5.a 所示；2）“合成的”，使用“纯净的”创建的，使用了上述的增强策略。它包含 200 个样本，其中一些如图 5.b 所示；3）“现实世界”，其中包含用手机相机拍摄的音乐书籍中的 200 张图像。例子如图 5.c 所示。 

由于我们的训练数据有限，因此我们使用简化的 CRNN 配置来减少模型容量。与表1 中指定的配置不同，我们移除了第 4 和第 6 卷积层，将 2 层双向 LSTM 替换为 2 层单向 LSTM。网络对图像对和对应的标签序列进行训练。使用两种方法来评估识别性能：1）片段准确度，即正确识别的乐谱片段的百分比；2）平均编辑距离，即预测音调序列与真实值之间的平均编辑距离。为了比较，我们评估了两种商用 OMR 引擎，即 Capella Scan 和 PhotoScore。    

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/9.jpg?raw=true)  

表 4 总结了结果。CRNN 大大优于两个商业系统。Capella Scan 和 PhotoScore 系统在干净的数据集上表现相当不错，但是它们的性能在合成和现实世界数据方面显著下降。主要原因是它们依赖于强大的二值化来检五线谱和音符，但是由于光线不良，噪音破坏和杂乱的背景，二值化步骤经常会在合成数据和现实数据上失败。另一方面，CRNN 使用对噪声和扭曲具有鲁棒性的卷积特征。此外，CRNN 中的循环层可以利用乐谱中的上下文信息。每个音符不仅自身被识别，而且被附近的音符识别。因此，通过将一些音符与附近的音符进行比较可以识别它们，例如对比他们的垂直位置。  

结果显示了 CRNN 的泛化性，因为它可以很容易地应用于其它的基于图像的序列识别问题，需要极少的领域知识。与 Capella Scan 和 PhotoScore 相比，我们的基于 CRNN 的系统仍然是初步的，并且缺少许多功能。但它为 OMR 提供了一个新的方案，并且在音高识别方面表现出有前途的能力。  

## 4. Conclusion  

在本文中，我们提出了一种新颖的神经网络架构，称为卷积循环神经网络（CRNN），其集成了卷积神经网络（CNN）和循环神经网络（RNN）的优点。CRNN 能够获取不同尺寸的输入图像，并产生不同长度的预测。它直接在粗粒度的标签（例如单词）上运行，在训练阶段不需要详细标注每一个单独的元素（例如字符）。此外，由于 CRNN 放弃了传统神经网络中使用的全连接层，因此得到了更加紧凑和高效的模型。所有这些属性使得 CRNN 成为一种基于图像序列识别的极好方法。  

在场景文本识别基准数据集上的实验表明，与传统方法以及其它基于 CNN 和 RNN 的算法相比，CRNN 实现了优异或极具竞争力的性能。这证实了所提出的算法的优点。此外，CRNN 在光学音乐识别（OMR）的基准数据集上显著优于其它的竞争者，这验证了 CRNN 的泛化性。  

实际上，CRNN 是一个通用框架，因此可以应用于其它的涉及图像序列预测的领域和问题（如汉字识别）。进一步加快 CRNN，使其在现实应用中更加实用，是未来值得探索的另一个方向。

# STN-OCR: A single Neural Network for Text Detection and Text Recognition  

## Abstract  

在自然场景图像中检测和识别文本是一项具有挑战性但尚未完全解决的任务。近年来，已经提出了几个试图解决两个子任务（文本检测和文本识别）中的至少一个的新系统。在本文中，我们介绍 STN-OCR，这是半监督神经网络用于场景文本识别的一个步骤，可以进行端到端优化。与大多数由多个深度神经网络和多个预处理步骤组成的现有作品相比，我们建议使用一个深度神经网络，通过半监督方式学习检测和识别自然图像中的文本。STN-OCR 是一个集成并共同学习空间变换网络的网络，可以学习检测图像中的文本区域，还可以使用文本识别网络来识别文本区域并识别它们的文本内容。我们研究我们的模型如何在一系列不同的任务（检测和识别字符以及文本行）上发挥作用。公共基准数据集上的实验结果表明，我们的模型能够处理各种不同的任务，而整个网络结构没有实质性变化。  

## 1. Introduction  

文字在我们的日常生活中无处不在。文字可以在文件，道路标志，广告牌和其他物体（如汽车或电话）上找到。从自然场景图像自动检测和阅读文本是系统的重要组成部分，可用于几项具有挑战性的任务，如基于图像的机器翻译，自动驾驶汽车或图像/视频索引。近年来，在自然场景中检测文本和识别文本的任务已经引起了计算机视觉和文档分析界的极大兴趣。此外，近期在计算机视觉领域取得的突破能够创造出比以前更好的场景文本检测和识别系统。虽然光学字符识别（OCR）的问题可以看作是解决了印刷文档文本，但检测和识别自然场景图像中的文本仍然具有挑战性。包含自然场景的图像表现出明显的光照变化，视角扭曲，图像质量，文本字体，不同背景等。  

现有的大部分研究工作都开发了端到端的场景文本识别系统，这些系统由复杂的两步流程组成，第一步是检测图像中的文本区域，第二步是识别该区域的文本内容。大部分现有作品只专注于这两个步骤中的一个。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/1.jpg?raw=true)  

在本文中，我们提出一个由单个深度神经网络（DNN）组成的解决方案，该网络可以以半监督的方式学习检测和识别文本。这与现有的作品是相反的，在这些作品中，文本检测和文本识别系统是以完全监督的方式单独训练的。最近的工作表明，卷积神经网络（CNN）能够学习如何解决复杂的多任务问题，同时接受端到端的训练。我们的动机是使用 CNN 的这些功能，并创建一个端到端的场景文本识别系统，通过将手头的任务分成较小的子任务并相互独立地解决这些子任务，从而表现得更像人。为了实现这种行为，我们学习了一个能够将输入图像划分为子任务（单个字符，单词或者甚至是文本行）的单个 DNN，并相互独立地解决这些子任务。这是通过联合学习使用循环空间转换器作为注意机制的定位网络和文本识别网络实现的（参见图 1 以获得系统的示意图）。在此设置中，网络仅接收图像和包含在该图像中的文本的标签作为输入。文本的定位由网络本身学习，使得这种方法是半监督的。  

我们的贡献如下：（1）我们提出的系统是通过集成空间变换器网络来解决端到端场景文本识别的一个步骤。（2）我们以半监督的方式端对端地训练我们提出的系统。（3）我们证明我们的方法能够在一系列标准场景文本检测和识别基准上达到最先进/最具竞争力的性能。（4）我们向研究界提供我们的 code 和训练好的模型。  

本文的结构如下：在第 2 节中，我们概述了与我们有关的其他研究人员的工作。第 3 节详细介绍了我们提出的系统，并提供了如何训练这种系统的最佳实践。我们在第 4 节展示并讨论标准基准数据集的结果，并在第 5 节总结我们的发现。  

## 2. Related Work  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/2.jpg?raw=true)  

多年来，已经开发和发布了不同场景文本检测和识别的方法。几乎所有的系统都使用两步过程来执行场景文本的端到端识别。第一步是检测文本区域并从输入图像中提取这些区域。第二步是识别文本内容并返回这些提取的文本区域的文本字符串。  

还可以将这些方法分为三大类：（1）依靠手工特征和人类知识进行文本检测和文本识别的系统。（2）使用深度学习方法的系统与手工制作的特征一起使用，或者对于两个步骤中的每一个使用两个不同的深度网络。（3）不是由两步法组成的系统，而是使用单个深度神经网络进行文本检测和识别。我们将在下面的每个类别中讨论一些这些系统。  

**Hand Crafted Features** 在开始的基础上，手工制作的特征和人类的知识已被用于执行文本检测。这些系统使用 MSER，Stroke Width Transforms  或 HOGFeatures 等特征来识别文本区域并将其提供给系统的文本识别阶段。在文本识别阶段使用滑动窗口分类器和 SVMs 或使用 HOG 特征的 k-最近邻分类器的集合。所有这些方法都使用手工制作的特征，这些特征具有各种各样的超参数，需要专业知识才能正确调整以获得最佳结果。  

**深度学习方法** 更新的方法在端到端的识别系统中使用 DNNs 来进行特征提取。G'omez 和 Karatzas 提出了一种特定文本的选择性搜索算法，与 DNN 一起可以用来检测自然场景图像中的（失真）文本区域。Gupta 等人提出了基于 YOLO 架构的文本检测模型，该模型使用全卷积深度神经网络来识别文本区域。这些方法识别的文本区域可以用作基于执行文本识别的 DNN 的其他系统的输入。  

Bissacco等人提出了一个完整的端到端体系结构，使用手工制作的特征执行文本检测。识别的文本区域被二值化，然后用作深度全连接的神经网络的输入，该神经网络将每个找到的字符独立分类。Jaderberg 等人提出了几个使用深度神经网络进行文本检测和文本识别的系统。Jaderberg 等人提出了一种滑动窗口文本检测方法，以多种分辨率在图像上滑动卷积文本检测模型。文本识别阶段使用单个字符 CNN，该字符滑过已识别的文本区域。此 CNN 与用于文本检测的 CNN 共享权重。Jaderberg 等人建议使用带有用于文本检测的额外边界框回归 CNN 的区域提议网络和以整个文本区域作为输入并且跨越预先定义的单词词典执行分类的 CNN，使得该方法仅适用于给定的语言。  

Goodfellow 等人提出了一个房屋号码的文本识别系统，Jaderberg 等人进行了优化，用于无约束的文本识别。该系统使用一个 CNN，它将完整提取的文本区域作为输入，并提供该文本区域中包含的文本。这是通过给定单词中每个可能的字符都有一个独立的分类器来实现的。基于这个想法，He等人和Shi 等人提出了文本识别系统，其将提取的文本区域中的字符识别视为序列识别问题。He 等人使用一种朴素的滑动窗口方法创建文本区域的切片，用作文本识别 CNN 的输入。由文本识别 CNN 产生的特征被用作预测字符序列的递归神经网络（RNN）的输入。在我们关于纯场景文本识别的实验中（参见第 4.3 节以获取更多信息），我们使用了类似的方法，但是我们的系统使用了更复杂的滑动窗口方法，其中滑动窗口的选择由网络自动学习。Shi 等人利用一个 CNN，它使用完整的文本区域作为输入，并产生一个特征向量序列，这些特征向量被馈送到预测提取的文本区域中的字符序列的 RNN。该方法基于文本区域的宽度生成固定数量的特征向量。这意味着对于仅包含几个字符但与具有足够多字符的文本区域相同的宽度的文本区域，该方法将产生用作 RNN 输入的相同数量的特征向量。在我们的纯文本识别实验中，我们利用我们的方法的优势来学习提取文本区域中最重要的信息，因此只需要生成尽可能多的特征向量。 Shi 等人通过首先增加额外的步骤来改善他们的方法，该步骤利用空间转换网络的整流能力来纠正提取的文本行。其次，他们在他们的网络中增加了一种软注意力机制，有助于在输入图像中产生字符序列。在他们的工作中 Shi 等人利用空间转换器作为额外的预处理步骤，使识别网络更容易识别图像中的文本。在我们的系统中，我们使用空间转换器作为以半监督方式检测文本的核心构建块。  

**端到端的可训练方法** 所提出的系统总是使用两步法来检测和识别来自场景文本图像的文本。尽管最近的方法利用深度神经网络，但他们仍然在任一步骤中或在两步骤的结果融合在一起的地方使用大量手工制作的知识。Smith 等人提出了一种端到端的可训练系统，该系统能够使用单个 DNN 检测和识别法国街道名称标志上的文本。与我们的系统不同，系统不提供图像中文本的位置，只能提取文本内容。此外，我们的方法中使用的注意力机制显示出更像人类的行为，因为它是依次定位并识别来自给定图像的文本。  

## 3. Proposed System  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/3.jpg?raw=true)  

人类会按顺序找到并阅读文本。第一个操作是将注意力放在一行文本上，顺序读取每个字符，然后再处理下一行文本。目前大多数用于场景文本识别的端到端系统都没有这种行为。这些系统试图通过一次从图像中提取所有信息来解决问题。我们的系统首先尝试顺序关注图像中的不同文本区域，然后识别每个文本区域的文本内容。为此，我们创建了一个简单的 DNN，包含两个阶段：（1）文本检测（2）文本识别。在本节中，我们将介绍文本检测阶段所使用的注意概念和所提议系统的总体结构。我们还报告了成功训练这种系统的最佳实践。  

### 3.1. Detecting Text with Spatial Transformers  

由 Jaderberg 等人提出的空间转换器是用于 DNN 的可微模块，其采用输入特征图 I 并将空间变换应用于该特征图，从而产生输出特征图 O。这样的空间变换模块由三部分组成。第一部分是一个定位网络，它计算一个函数 $f_{loc}$，它预测要应用的空间变换的参数 $\theta$。在第二部分中使用这些预测参数来创建采样网格，该采样网格定义输入特征映射的哪些特征应映射到输出特征映射。第三部分是一种可微插值方法，它将生成的采样网格生成为空间变换的输出特征映射 O。我们将在下面的段落中简要描述每个组件。  

**定位网络** 定位网络采用输入特征映射 $I \in R^{C \times H \times W}$，C 通道，高度 H和宽度 W，并输出应用变换的参数 $\theta$。在我们的系统中，我们使用定位网络（$f_{loc}$）来预测 N 个二维仿射变换矩阵 $A_{\theta}^n$，其中 $n \in \{0,...,N-1\}$：  

$f_{loc}(I) = A_{\theta}^n = \begin{bmatrix} \theta_1^n & \theta_2^n & \theta_3^n \\ \theta_4^n & \theta_5^n & \theta_6^n \\ \end{bmatrix} \tag{1}$

因此 N 是定位网络定位的字符，词语或文本线的数量。以这种方式预测的仿射变换矩阵允许网络对输入图像应用平移，旋转，缩放和倾斜，因此网络学习产生变换参数，该变换参数可以放大将要从输入图像中提取的字符，单词或文本行图片。  

在我们的系统中，通过使用前馈 CNN 和 RNN 一起产生 N 个变换矩阵 $A_{\theta}^n$。对于 RNN 的每个时间步骤，使用隐藏状态 $h_n$ 来计算 N 个变换矩阵中的每一个：  

$c = f_{loc}^{conv}(I) \tag{2}$
$h_n = f_{loc}^{rnn}(c,h_{n-1}) \tag{3}$
$A_{\theta}^n = g_{loc}(h_n) \tag{4}$

其中 $g_{loc}$ 是另一个前馈网络，并且每个变换矩阵 $A_{\theta}^n$ 是以全局提取的卷积特征（$f_{loc}^{conv}$）和以先前执行的时间步骤的隐藏状态为条件计算的。  

我们使用的定位网络中的 CNN 是 He等人著名的 ResNet 的变体。我们使用 ResNet 的一个变种，因为我们发现，与使用 VGGNet 等其他网络结构的实验相比，通过这种网络结构，我们的系统学得更快，更成功。我们认为这是由于 ResNet 的残差连接有助于保持强梯度下降到最初的卷积层的事实。除了结构之外，我们还对所有实验使用了批量标准化。定位网络中使用的 RNN 是双向长短时记忆（BLSTM）单元。这个 BLSTM 用于生成隐藏状态 $h_n$，而隐藏状态 $h_n$ 又用于预测仿射变换矩阵。我们在第 4 节中报告的所有实验中使用了相同的网络结构。图 3 提供了该网络的结构概述。  

**网格生成器** 网格生成器使用具有高度 $H_o$ 和宽度 $W_o$ 的坐标 $y_{h_o}, x_{w_o}$和仿射变换矩阵 $A_{\theta}^n$ 的规则间隔网格 $G_o$ 来生成输入 N 个规则网格 $G_i^n$,将特征图 I 的坐标映射为 $u_i^n, v_j^n$，其中 $i \in H_o, j \in W_o$：  

$ \begin{pmatrix} u_i^n \\ v_j^n \\ \end{pmatrix} = A_{\theta}^n \begin{pmatrix} x_{w_o} \\ y_{h_o}\\ 1 \end{pmatrix} = \begin{bmatrix} \theta_1^n & \theta_2^n & \theta_3^n \\ \theta_4^n & \theta_5^n & \theta_6^n \\ \end{bmatrix} \begin{pmatrix} x_{w_o} \\ y_{h_o}\\ 1 \end{pmatrix} \tag{5}$

在推理过程中，我们可以提取 N 个结果网格 $G_i^n$，其中包含定位网络找到的文本区域的边界框。高度 Ho 和宽度 Wo 可以自由选择，如果它们低于输入特征图 I 的高度 H 或宽度 W，则网格生成器生成网格，该网格在下一步中执行下采样操作。  

**图像采样** 网格生成器生成的 N 个采样网格 $G_i^n$ 现在用于对每个 $n \in N$ 在对应的坐标 $u_i^n, v_j^n$ 上采样特征图 I。当然，这些点在输入特征图中并不总是与离散网格完全对齐的。因此，我们使用双线性采样，通过对最近邻居的值进行双线性内插来提取给定坐标处的值。由此，我们定义 给定位置 i,j 上 N 个输出特征图 $O^n$ 的值，其中 $i \in H_o, j \in W_o$：  

$O_{i,j}^n = \sum_h^{H_o} \sum_w^{W_o} I_{hw} max (0, 1-|u_i^n -h|) max (0,1 - |v_j^n - w|)$

这种双线性采样是（次）可微分的，因此可以通过使用标准反向传播将误差梯度传播到定位网络。  

定位网络，网格生成器和图像采样器的组合形成空间变换器，并且通常可以用于 DNN 的每个部分。在我们的系统中，我们使用空间变换器作为我们网络的第一步。定位网络接收输入图像作为输入特征映射，并产生一组仿射变换矩阵，由网格生成器用来计算应由双线性采样操作采样的像素的位置。  

### 3.2. Text Recognition Stage  

文本检测阶段的图像采样器产生从原始输入图像提取的一组 N 个区域。文本识别阶段（这个阶段的结构概述可以在图 3 中找到）使用这 N 个不同区域中的每一个并且彼此独立地处理它们。N 个不同区域由 CNN 处理。这个 CNN 也基于 ResNet 架构，因为我们发现如果我们使用 ResNet 架构的变体来实现我们的识别网络，我们能取得好的结果。我们认为在识别阶段使用 ResNet 比在检测阶段更重要，因为检测阶段需要从识别阶段接收强梯度以成功更新定位网络的权重。识别阶段的 CNN 预测标签空间 $L_{\epsilon}$ 上的概率分布 $\hat y$，其中 $L_{\epsilon} = L \bigcup \{\epsilon\}$，其中 L = {0-9a-z}，$\epsilon$ 代表空白标签。根据任务的不同，这种概率分布或者由固定数量的 T softmax 分类器生成，其中每个 softmax 分类器用于预测给定单词的一个字符：  

$ \begin{align}
x^n = O^n \tag{7} \\
\hat y_t^n = softmax(f_{rec}(x^n)) \tag{8} \\
\hat y^n = \sum_{t=1}^T \hat y_t^n \tag{9} \\
\end{align} $

其中 $f_{rec}(x)$ 是在采样输入 x 上应用卷积特征提取器的结果。  

另一种可能性是使用 Connectionist Temporal Classification（CTC）对网络进行训练，并通过将 $\hat y$ 设置为最可能的标记路径 $\pi$ 来检索最可能的标记，其由下式给出：  

$ \begin{align}
p(\pi|x^n) = \quad_{t=1}^{T} \hat y_{\pi t}^n, \ \pi \in L_{\epsilon}^T \tag{10} \\
\hat y_t^n = argmax p(\pi|x^n) \tag{11} \\
\hat y^n = B (\sum_{t=1}^T \hat y_t^n) \tag{12} \\
\end{align} $

其中 $L_{\epsilon}^T$ 是所有长度为 T 的标签的集合，$p(\pi|x^n)$ 是 DNN 预测路径 $\pi \in L_{\epsilon}^T$ 的概率。B 是除去所有预测空白标签和所有重复标签（例如 B(-IC-CC-V)= B(II-CCC-C-V-) = ICCV）的函数。  

### 3.3. Model Training  

用于训练模型的训练集 X 包括一组输入图像 I 和一组用于每个输入图像的文本标签 $L_I$。我们不使用任何标签来训练文本检测阶段。这个阶段仅通过计算预测标签和文本标签的交叉熵损失或 CTC 损失获得的误差梯度来学习检测文本区域。实验中，我们发现，从头开始训练时，检测并识别两条以上文本行的网络不会收敛。解决这个问题的方法是在难度逐渐增加的情况下执行一系列预训练步骤。此外，我们发现选择用于训练网络的优化算法对网络的收敛性有很大影响。我们发现使用随机梯度下降（SGD）对网络进行简单任务训练是有益的，Adam 利用更多的文本线对图像上已经预先训练好的网络进行微调。我们认为 SGD 在预训练期间表现更好，因为学习率 η 在较长的时间内保持不变，这使得文本检测阶段能够探索输入图像并更好地查找文本区域。随着学习速度的降低，检测阶段的更新变得更小，并且文本检测阶段（理想地）稳定在已经找到的文本区域上。同时，文本识别网络可以开始使用提取的文本区域并学习识别该区域中的文本。在使用 SGD 对网络进行训练时，需要注意的是，选择过高的学习率会导致模型早期出现分歧。我们发现，使用 $1^{-5}$ 和 $1^{-7}$ 之间的初始学习率几乎适用于所有情况，除非网络应该进行微调。在这里，我们发现使用 Adam 是更可靠的选择，因为 Adam 以自适应的方式选择每个参数的学习速率。  

## 4. Experiments  

在本节中，我们将在几个标准场景文本检测/识别数据集上评估我们提出的网络体系结构。我们给出三个不同数据集的实验结果，其中每个数据集的任务难度增加。我们的实验首先从 SVHN 数据集开始，用来证明我们的概念是可行的。我们进行实验的第二种数据集是用于聚焦的场景文本识别的数据集，在这里探索了我们的模型的性能，当涉及到查找和识别单个字符时。我们欣赏的第三个数据集是法国街道名称符号（FSNS）数据集，这是我们使用的最具挑战性的数据集，因为此数据集包含大量不规则的低分辨率文本行。通过介绍我们的实验设置开始本节。然后，将介绍上述每个数据集的实验结果和特征。  

### 4.1. Experimental Setup  

**定位网络** 每个实验中使用的定位网络都基于 ResNet 架构。网络的输入是文本应该被定位并且被识别的图像。在第一个残差块之前，网络执行 3×3 卷积，然后执行 2×2 平均池化，步长为 2。在这些层之后，使用了三个具有两个 3×3 卷积的残差块，每个卷积都采用批量归一化。卷积滤波器的数量分别为 32，48 和 48，并且 ReLU 被用作每个卷积层的激活函数。在第二个残差块之后跟随一个带有步幅 2 的 2×2 最大池化。最后一个残差块后面跟着一个 5×5 的平均池化层，这个层跟着一个 256 个隐藏单元的 BLSTM。对于 BLSTM 的每个时间步骤，都有一个带有 6 个隐藏单元的全连接层。该层预测仿射变换矩阵，用于生成双线性插值的采样网格。由于场景文本的修正超出了本文的范围，因此我们通过将相关参数设置为0来禁用仿射变换矩阵中的偏斜和旋转。我们将在以后的工作中讨论空间变换器对场景文本检测的纠正能力。  

**识别网络** 识别网络的输入是来自原始输入图像的 N 个 crops，代表定位网络找到的文本区域。识别网络具有与定位网络相同的结构，但卷积滤波器的数量更多。卷积滤波器的数量分别是 32，64 和 128。取决于实验，我们使用[6]和[17]中使用的 T 个独立的 softmax 分类器的集合，其中 T 是单词可能具有的最大长度，或者我们使用[11]中所用的具有最佳路径解码的 CTC。  

**实现** 我们使用 MXNet 实现了所有的实验。我们在一台拥有英特尔®酷睿™i7-6900K CPU，64 GB RAM 和 4颗TITAN X（Pascal）GPU 的工作站上进行了所有实验。  

### 4.2. Experiments on the SVHN dataset  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/8.jpg?raw=true)  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/4.jpg?raw=true) 

通过我们在 SVHN 数据集上的第一次实验，我们想证明我们的概念是有效的，并且可以用于现实世界的数据。因此，我们首先在 SVHN 图像上进行了与[16]中的实验相似的实验，每个图像中都有一个门牌号码，以数字为中心并且还包含背景噪音。表 1 显示我们能够达到有竞争力的识别精度。  

基于这个实验，我们想要确定我们的模型是否能够检测排列成规则网格或放置在图像中随机位置的不同文本行。在图 4 中，我们展示了来自为两个目的构建的数据集中的样本，我们将其用于基于 SVHN 数据的其他实验。我们发现，我们的网络在寻找和识别以规则网格排列的房屋号码的任务方面表现良好。我们在对这些数据进行训练期间所做的一个有趣的观察是，当我们进行了两个训练步骤时，我们能够取得最佳成果。第一步是从头开始训练整个模型（所有权重都是随机初始化的），然后再次用相同的数据训练模型，但是这次定位网络使用从上一次训练获得的权重进行初始化，识别网络用随机权重初始化。该策略导致定位网络更好的定位结果，并因此改善识别结果。  

在我们对由我们创建的第二个数据集进行的实验中，我们发现从零开始训练模型是不可能的，它可以找到并识别散布在整个图像上的两个以上的文本行。 通过首先在较容易的任务上训练模型（几条文本线，接近图像中心的文本线），然后逐渐增加任务的难度，可以训练这样的网络。在补充材料中，我们提供了简短的视频剪辑，显示了网络如何探索图像，同时学习检测各种不同实验的文本。  

### 4.3. Experiments on Robust Reading Datasets  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/5.jpg?raw=true) 
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/6.jpg?raw=true) 

在我们的下一个实验中，我们使用了从输入图像中裁剪文本区域的数据集。我们想看看我们的文本定位网络是否可以用作智能滑动窗口生成器，该生成器采用裁剪文本区域中文本的不规则性。因此，我们使用 CTC 在我们自己的数据生成器生成的合成裁剪字图像数据集上训练了我们的识别模型，该数据生成器与 Jaderberg 等人引入的数据生成器类似。在表 2 中，我们报告了我们的模型在 ICDAR 2013 robust reading，街景文本（SVT）和 IIIT5K 基准数据集上的识别结果。为了对 ICDAR 2013 和 SVT 数据集进行评估，我们过滤了所有包含非字母数字字符的图像，并丢弃了[28,32]中所做的小于 3 个字符的所有图像。我们通过使用标准 hunspell english（en-US）字典后处理预测来获得最终结果。总体而言，我们发现我们的模型在 ICDAR 2013 和 IIIT5K 数据集上获得了无约束识别模型的最先进性能，并在 SVT 数据集上实现了竞争性能。在图 5 中，我们显示我们的模型学习遵循单个文本区域的斜率，证明我们的模型以智能方式生成滑动窗口。  

### 4.4. Preliminary Experiments on the FSNS dataset  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/7.jpg?raw=true) 

根据我们提高网络解决任务难度的方案，我们选择了 Smith 等人的法国街道名称标志（FSNS）数据集，成为我们的第三个数据集来进行实验。我们在这里报告的结果是初步的，仅表明我们的网络架构也适用于这类数据，尽管它尚未达到最新的结果。FSNS 数据集包含从 Google Streetview 中提取的法国街道名称标志的图像。这个数据集对于我们的方法来说是最具挑战性的数据集，因为它（1）包含嵌入自然场景中具有不同长度的多行文本，具有分散的背景和（2）包含大量不包含街道全名的图像。  

在我们对该数据集进行的第一次实验中，我们发现我们的模型在提供的 groundtruth 上训练时不能收敛。我们认为这是因为原始数据集的标签不包含任何关于哪些文字可以在哪个文本行中找到的提示。因此，我们改变了方法，并开始尝试寻找单个单词而不是多个单词的文本行。我们相应地修改了 groundtruth，并将包含最多三个词的所有图像用于我们的实验，这为我们留下了大约 80% 的来自数据集的原始数据。图 6 显示了 FSNS 数据集中的一些示例，我们的模型正确定位了单个单词并正确识别了这些单词。使用这种方法，我们能够在测试集上获得 97% 的合理良好的字符识别准确性，但只有71.8% 的字词准确率。字符识别率和字识别率的差异是由于我们为这个任务训练的模型使用独立的 softmax 分类器来处理单词中的每个字符。具有 97% 的字符识别准确度意味着至少一个分类器出错并且因此增加序列错误的可能性很高。  

## 5. Conclusion  

在本文中，我们提出了一个系统，该系统可以看作是仅使用单个多任务深度神经网络来解决端到端场景文本识别的一个步骤。我们以半监督的方式训练了我们模型的文本检测组件，并且能够提取文本检测组件的定位结果。我们的系统的网络架构很简单，但要训练这个系统并不容易，因为成功的训练需要在模型能够收敛到真正的任务之前对更简单的子任务进行大量的预训练。我们还表明，可以使用相同的网络架构在一系列用于场景文本检测/识别的公共基准数据集上获得竞争性或最新的结果。  

在当前状态下，我们注意到我们的模型不能完全检测图像中任意位置的文本，正如我们在 FSNS 数据集实验中看到的那样。目前，我们的模型还受限于可以一次检测到的固定数量的最大文本行/字符，在我们未来的工作中，我们希望以一种方式重新设计网络，使网络能够确定文本行的数量 一个图像本身。

# Synthetic Data for Text Localisation in Natural Images  

## Abstract  

在本文中，我们介绍一种用于自然图像中文本检测的新方法。该方法包括两个贡献：首先，一个快速和可扩展的引擎，用于生成杂乱的文本合成图像。该引擎以自然的方式将合成文本叠加到现有背景图像上。其次，我们使用合成图像来训练全卷积回归网络（FCRN），该网络可高效地在图像中的所有位置和多个比例上执行文本检测和边界框回归。我们讨论了 FCRN 与最近推出的 YOLO 检测器以及其他基于深度学习的端到端物体检测系统的关系。由此产生的检测网络大大超出了自然图像中文本检测的当前方法，在标准 ICDAR 2013 基准测试中实现了 84.2% 的 F 值。此外，它可以在 GPU 上每秒处理 15 个图像。  

## 1. Introduction  

文本识别，即在自然场景中阅读文本的能力，在计算机视觉的以人类为中心的应用中是非常令人满意的特征。最先进的系统通过结合两个简单而强大的见解，实现了高文本识别性能。首先是复杂的识别流程，通过明确地结合识别和检测单个字符来识别文本，可以被非常强大的分类器所替代，这些分类器可以直接将图像 patches 映射到单词。第二，这些强大的分类器可以通过综合生成所需的训练数据来学习。  

虽然文献[20]成功地解决了在给定包含单词的图像 patches 的情况下识别文本的问题，但是获取这些 patches 的过程仍然不理想。该流程结合了 HoG ，EdgeBoxes 和 Aggregate Channel Features 等通用功能，并且仅在后期阶段才引入文本特定（CNN）特征，其中最终将 patches 识别为特定词。由于两个原因，这种情况非常不可取。首先，检测流程的性能成为文本识别的新瓶颈：在[20]中正确裁剪的词语的识别准确率为98%，而端到端的文本识别 F 分数仅为 69%，主要是由于不正确的或错过了字区域建议。其次，流程缓慢而不雅。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/1.jpg?raw=true)  
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/2.jpg?raw=true)  

在本文中，我们提出类似于[20]的文本检测问题的补充改进。我们做出两项重要贡献。首先，我们提出了一种用于生成合成图像的新方法，该方法自然地融合了现有自然场景中的文本，使用现成的深度学习和分割技术将文本与背景图像的几何图形对齐，并尊重场景边界。我们使用这种方法**在杂乱的条件下自动生成一个新的综合文本数据集**（图1（上）和第 2 节）。该数据集称为 Wild 中的 SynthText（图2），适用于训练高性能场景文本检测器。与现有合成文本数据集（如[20]之一）的主要区别在于，它们仅包含字级图像区域，不适合训练检测器。  

第二个贡献是**深层文本检测架构**，既精确又高效（图1（下）和第 3 节）。我们称之为全卷积回归网络。与用于图像分割的全卷积网络（FCN）模型类似，它在每个图像位置密集地执行预测。然而，与 FCN 不同的是，预测不仅仅是一个分类标签（文本/不是文本），而是包围以该位置为中心的单词的边界框的参数。后者的想法是借鉴了 Redmon 等人的 You Only Only Once（YOLO）技术，但卷积回归机显着提升了性能。  

新的数据和检测器在标准基准数据集（第4部分）上实现了最先进的文本检测性能，同时在测试时间比传统文本检测器快一个数量级（GPU 上每秒高达 15 幅图像）。我们还证明了在数据集中逼真性的重要性，通过显示如果检测器在图像上用综合插入的词汇进行训练而不考虑场景布局，则检测性能明显较差。 最后，由于更精确的检测步骤，一旦新检测器换入现有流程中，端到端的单词识别也得到改进。我们的研究结果在第 5 部分中总结。  

### 1.1. Related Work  

**使用 CNN 进行对象检测** 我们的文本检测网络主要基于 Long 等人的全卷积网络和 Redmon 等人的基于 YOLO 图像网格的边界框回归网络。YOLO 是使用 CNN 特征进行对象类别检测的一个广泛工作的一部分，可追溯到 Girshick 等人的 Region-CNN（R-CNN）框架区域提案和 CNN 特征的组合。R-CNN 框架有三个广泛的阶段 - （1）生成对象提议，（2）提取每个提议的 CNN 特征映射，以及（3）通过特定分类的 SVM 过滤提议。Jaderberg 等人的文本识别方法也使用类似的流程进行检测。Girshick 等人将独立的提取每个区域的特征图确定为瓶颈。在 Fast R-CNN中，他们通过计算一次 CNN 特征并为每个提案汇集它们，从而在 R-CNN 上获得 100 倍的加速；他们还将 R-CNN 的最后两个阶段简化为单个多任务学习问题。这项工作揭示了区域建议阶段是新的瓶颈。Lenc 等人完全放弃区域建议阶段，并使用 PASCAL VOC 数据上通过 K 均值聚类学习的恒定区域集合。Ren 等人也从一套固定的提案开始，但在检测之前通过使用区域提案网络进行改进，该提案网络与后来的检测网络分享权重并简化多阶段 R-CNN 框架。  

**合成数据** 合成数据集提供详细的真实注释，并且是手动标注图像的便宜且可扩展的替代方案。他们已被广泛用于学习大型 CNN 模型 - Wang 等人和Jaderberg 等人使用合成文本图像来训练词语图像识别网络；Dosovitskiy 等人使用浮椅渲染来训练密集的光流回归网络。详细的合成数据也被用于学习生成模型 - Dosovitskiy 等人训练 inverted CNN 模型来呈现 chairs 的图像，而 Yildirim 等人使用在合成脸部渲染上训练的深 CNN 特征来从脸部图像中回归姿态参数。  

**Augmenting 单个图像** 有一大部分工作是以真实的方式插入物体，并从单幅图像中推断 3D 结构 - Karsch 等人开发了一种令人印象深刻的半自动方法来渲染具有正确照明和视角的物体；他们根据 Criminisi 等人的技术推断物体的实际尺寸。Hoiem 等人将图像区域从单个图像分为地平面，垂直平面或天空，并通过将图像分解为平面来生成“pop-ups”。同样，我们也将单个图像分解成局部平面区域，但使用 Liu 等人的密集深度预测。 

## 2. Synthetic Text in the Wild  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/3.jpg?raw=true)  

包含数百万个参数的深度 CNN 等大型模型的监督训练需要非常大量的标记训练数据，这对手动获取来说是昂贵的。此外，如表 1 所总结的，公开可用的文本发现或检测数据集非常小。这些数据集不仅不足以训练大型 CNN 模型，还不足以表示自然场景中字体，颜色，大小，位置等可能的文本变化空间。因此，在本节中，我们开发了一个合成文本场景图像生成引擎，用于构建用于文本定位的大型标注数据集。  

我们的合成引擎（1）生成逼真的场景文本图像，以便训练过的模型可以推广到真实（非合成）图像，（2）是全自动的，（3）快速，能够在无监督下生成大量的数据。文本生成流程可以概括如下（另见图3）。在获取合适的文本和图像样本（2.1节）后，图像被分割成基于局部颜色和纹理提示的连续区域，并且使用[30]的 CNN 获得密集的像素深度图（2.2节）。然后，对于每个连续区域估计局部表面法线。  

接下来，根据区域的颜色（第2.3节）选择文本的颜色以及可选的轮廓颜色。最后，使用随机选择的字体呈现文本样本并根据局部表面方向进行变换；使用泊松图像编辑将文本混合到场景中。我们的引擎需要大约半秒来生成新的场景文本图像。  

该方法用于生成 800,000 个场景文本图像，每个图像具有以不同样式呈现的多个单词实例，如图2 所示。数据集可在以下网址获得：http://www.robots.ox.ac.uk/~vgg/data/scenetext/  

### 2.1. Text and Image Sources  

合成文本生成过程首先对一些文本和背景图像进行采样。该文本以三种方式从 Newsgroup20 数据集中提 -文字，行（最多 3 行）和段落（最多 7 行）。 单词被定义为由空格字符分隔的标记，线由换行符分隔。这是一个丰富的数据集，英文文本的自然分布中散布着符号，标点符号，名词和数字。  

为了支持多样化，通过与不同对象/场景以及室内/室外和自然/人工场所相关的查询从 Google 图像搜索中提取 8,000 个背景图像。为了保证所有的文本事件都被充分注释，这些图像不能包含它们自己的文本（街景文本的局限性在于注释并非详尽无遗）。因此，避免了会召回图像中大量文本的关键词（例如“街道标志”，“菜单”等） 包含文字的图像通过人工检查被丢弃。  

### 2.2. Segmentation and Geometry Estimation  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/4.jpg?raw=true)  

在实际图像中，文本倾向于包含在明确定义的区域（例如符号）中。我们通过要求文本被包含在具有统一颜色和纹理特征的区域中来逼近这个约束。这也可以防止文字跨越强烈的图像不连续性，这在实践中不太可能发生。使用[3]的有效图形切割实现，通过在 0.11 处对 gPb-UCM 轮廓分层结构[2]进行阈值化来获得区域。图 4 显示了一个关于局部区域提示的文本示例。  

在自然图像中，文字往往被涂在表面的顶部（例如标志或杯子）。为了近似我们的合成数据中的类似效应，文本根据局部表面法线进行透视变换。首先使用[30]的 CNN 预测密集的深度图，对上面分割的区域自动估计法线，然后使用 RANSAC 拟合一个平面的面。  

文本与估计的区域方向对齐如下：首先，使用估计的平面法线将图像区域轮廓翘曲为正视平面视图；然后，一个矩形适合于平行的区域；最后，文本与此矩形的较大边（“宽度”）对齐。 在同一区域放置多个文本实例时，将检查文本遮罩是否相互碰撞，以避免将它们放置在彼此之上。  

并非所有的分割区域都适合文本放置 - 区域不应该太小，具有极端的纵横比，或者表面法线与观察方向正交；所有这些区域都在这个阶段被过滤。此外，纹理过多的区域也被过滤，纹理的程度由 RGB 图像中三阶导数的强度来度量。  

**讨论** 使用 CNN 来估计深度（这是容易出错的过程）的替代方案是使用 RGBD 图像的数据集。我们宁愿估算不完美的深度图，因为：（1）它基本上允许使用任何场景类型的背景图像，而不仅仅是那些 RGBD 数据可用的背景图像；（2）因为公开可用的 RGBD 数据集，例如 NYUDv2，B3DO，Sintel 和 Make3D 在我们的上下文中有几个限制：小尺寸（NYUDv21 中 1500张图片，Make3D 中 400 张图片，B3DO 和 Sintel 中少量视频） ，低分辨率和运动模糊，对室内图像（在 NYUDv2 和 B3DO中）的限制，以及基于视频数据集（B3DO 和 Sintel）的图像的有限可变性。  

### 2.3. Text Rendering and Image Composition  

一旦文字的位置和方向被确定，文本就被分配一种颜色。文本的调色板是从 IIIT5K 单词数据集中的裁剪后的单词图像中学习的。使用 K 均值将每个裁剪后的单词图像中的像素分为两组，其中一个颜色接近前景（文本）颜色，另一个颜色背景。在渲染新文本时，选择其背景色与目标图像区域最匹配的颜色对（使用 Lab 颜色空间中的 L2 范数），并使用相应的前景色来渲染文本。  

大约 20% 的文本实例被随机选择为有边框。边框颜色选择为与前景色相同，其值通道增加或减少，或选择为前景色和背景色的平均值。  

为了保持合成文本图像中的亮度梯度，我们使用泊松图像编辑将文本混合到基本图像上，引导字段在其方程式（12）中定义。我们使用 Raskar1 提供的实现高效地解决此问题。  

## 3. A Fast Text Detection Network  

在本节中，我们将介绍用于自然场景中文本检测的 CNN 体系结构。虽然现有的文本检测流程结合了几个 ad-hoc 步骤并且速度很慢，但我们提出了一种高度准确，快速并可端到端训练的检测器。  

设 x 表示图像。基于 CNN 的检测最常见的方法是提出一些可能包含目标对象（我们的案例中的文本）的图像区域 R，裁剪图像，并使用 CNN 来评分它们是否正确。这种已被 R-CNN 广泛使用的方法运行良好，但速度很慢，因为它需要用 CNN 对每张图像评估数千次。  

另一种快速的对象检测策略是构造一个预测器的固定 field $(c,p) = \phi_{uv}(x)$，其中每一个都专门用于预测围绕特定图像位置(u,v)的对象是否 $c \in R$ 并且构成 $p =(x-u,y- v,w,h)$ 。这里姿势参数 (x,y) 和 (w,h) 分别表示紧密包围对象的边界框的位置和大小。  

虽然这种结构听起来很抽象，但它实际上是一种常见的结构，例如由隐式形状模型（ISM）和 Hough voting 实现。有一个预测器 $\phi_{uv}$ 查看以 (u,v) 为中心的局部图像片，并试图预测 (u,v) 周围是否有对象，以及对象相对于它的位置。  

在本文中，我们提出 Hough voting 的一个极端变体，受 Long 等人的全卷积网络（FCN）和 Redmon 等人的 You Only Only Once（YOLO）技术的启发。 。在 ISM 和 Hough 投票中，个人预测通过投票方案在图像中汇总。YOLO 类似，但避免投票并直接使用个人预测;因为这个想法可以加速检测，所以我们在这里采用它。  

YOLO 和 Hough 投票的另一个关键概念区别在于，霍夫投票预测变量 $\phi_{uv}$ 是局部和平移不变的，而在 YOLO 中它们不是：首先，在 YOLO 中，每个预测变量允许从整个图像中搜集证据，而不是只在一个以 (u,v) 为中心的图像 patch 中。其次，在不同位置 $(u,v \neq (u',v'))$ 的 YOLO 预测因子中是独立学习的不同函数 $\phi_{uv} \neq \phi_{u'v'}$。  

尽管 YOLO 的方法允许该方法获取用于检测 PASCAL 或 ImageNet 对象的上下文信息，但我们发现这不适用于更小和更可变的文本。相反，我们在这里提出一种在 YOLO 和 Hough 投票之间的方法。与在 YOLO 中一样，每个检测器 $\phi_{uv}(x)$ 仍然直接预测对象发生，而不经历昂贵的投票累积过程；然而，如在霍夫投票中，检测器 $\phi_{uv}(x)$ 是局部和平移不变的，共享参数。我们将这个平移不变和局部预测器的领域作为深度 CNN 的最后一层的输出来实现，从而获得全卷积回归网络（FCRN）。  

### 3.1. Architecture  

本节介绍 FCRN 的结构。首先，我们描述该体系结构的前几层，它计算文本特定的图像特征。然后，我们描述了建立在这些特征之上的密集回归网络，最后描述了它在多个尺度上的应用。  

**单尺度特征** 我们的架构受到 VGG-16 的启发，使用几层小密集滤波器；然而，我们发现一个更小的模型对文本的工作效果也更好。该体系结构包含 9 个卷积层，每个卷积层后面跟着 Relu，偶尔还有一个 Maxpooling 层。所有线性滤波器都有 1 个样本的步幅，并通过零填充来保留特征映射的分辨率。 Max-pooling 在 2×2 窗口上执行，步长为 2 个采样，因此特征图分辨率减半。  

**类和边界框预测** 单尺度特征终止于密集特征 field。假设有四个下采样 Maxpooling 图层，这些特征的步幅是 Δ= 16个像素，每个包含 512 个特征通道 $\phi_{uv}(x)$（为了方便起见，我们用 uv 表示像素）。  

给定特征 $\phi_{uv}(x)$，我们现在可以讨论构造密集文本预测器 $\phi_{uv}(x) = \phi_{uv}^r o \phi^f(x)$。这些预测因子被实现为另外七个 5×5 线性滤波器（C-7-5×5）$\phi_{uv}^r$，每个回归七个数字中的一个：对象存在置信度 c 和多达六个对象姿态参数 $p = (x-u, y-v, w, h, \cos \theta, \sin \theta)$ 其中 x，y，w，h 已经在前面讨论过，$\theta$ 是边界框旋转。  

因此，对于尺寸为 H×W 的输入图像，我们获得 $\frac {H}{\Delta} \times \frac {W}{\Delta}$ 个预测结果，每个结果对应于尺寸为 Δ×Δ 像素的图像单元。如果单词中心落入相应的单元格内，每个预测器负责检测一个单词。 YOLO 类似，但以大约一半的分辨率运行；更密集的预测器采样对于减少冲突（多个词落入同一单元格）并因此增加召回（因为每个单元最多可检测一个单词）是重要的。在实践中，对于 224×224 图像，我们获得 14×14 个cells/预测。  

**多尺度检测** 我们的卷积滤波器的有限接收域禁止检测大型文本实例。因此，我们得到输入图像的多个缩小版本的检测结果，并通过非极大值抑制来合并它们。更详细地说，输入图像通过因子 {1，1 / 2,1 / 4,1 / 8} 进行缩小（由于基线特征已经非常密集地计算，所以放大是过度的）。然后，通过抑制分数低于重叠检测的分数来合并所得到的检测结果。  

**训练损失** 我们对 CNN 的 $\frac {H}{\Delta} \times \frac {W}{\Delta} \times 7$ 中的每个输出使用一个平方损失项，与 YOLO 相似。如果单元格不包含真实词，则损失除了 c(text/no-text) 之外，忽略所有参数。  

**与 YOLO 比较** 我们的全卷积回归网络（FCRN）比 YOLO 网络（最后两个全连接层具有约 90% 的参数）少 30 倍的参数。由于其全局性，对于每个图像尺寸（包括多个尺度），标准 YOLO 必须进行再训练，进一步增加模型尺寸（而我们的模型需要 44MB，YOLO 需要 2GB）。这使 YOLO 不仅更难训练，而且效率也更低（比 FCRN 慢 2 倍）。  

## 4. Evaluation  

首先，在 4.1 节中，我们描述了我们评估模型的文本数据集。接下来，我们在 4.2 节的文本定位任务中评估我们的模型。在第 4.3 节中，为了研究合成数据生成流程中的哪些组件是重要的，我们执行详细的消融实验。在第 4.4 节中，我们使用定位模型的结果进行端到端的文本识别。我们在文本定位和端到端文本识别方面展现了实质性的改进。最后，在第 4.5 节中，我们将讨论使用我们的文本定位模型获得的加速。  

### 4.1. Datasets  

我们使用标准基准评估我们的文本检测网络：ICDAR 2011,2013 数据集和街景视图文本数据集。这些数据集将在下面进行回顾，他们的统计数据见表 1。  

**SynthText in the Wild** 这是使用我们的第 2 部分的合成引擎生成的 800,000 个训练图像的数据集。每个图像都有大约 10 个用字符和字级边界框注释的单词实例。  

**ICDAR 数据集** ICDAR 数据集（IC011，IC013）分别来自于 2011 年和 2013 年举办的 Robust 阅读挑战赛。它们包含标志牌，书籍，海报和其他具有world-level 轴对齐边界框注释的对象上的真实世界的文本图像。数据集大部分包含相同的图像，但是 shuffle 测试和训练集。我们不评估最近的 ICDAR 2015 数据集，因为它与 2013 数据集几乎相同。  

**街景视图文本** 此数据集缩写为 SVT，由谷歌街景收获的图像组成，该图像使用 word-level axis-aligned 边界框注释。SVT 比 ICDAR 数据更具挑战性，因为它包含更小和更低的分辨率文本。此外，并非所有文本实例都被注释。实际上，这意味着在评估过程中精确度被严重低估。为每幅图像提供包含 50 个干扰词以及真实词的词典；我们在 SVT 测试时将这些词典作为参考 。  

## 4.2. Text Localisation Experiments  

我们评估我们的检测网络：（1）比较应用于图像的单尺度和多尺度缩小版本时的性能；（2）使用高质量的区域提议，改进文本检测中的最新结果。  

**训练** FCRN 在 Wild 数据集的 SynthText 中对 800,000 个图像进行训练。每个图像的大小调整为 512×512 像素。在每个卷积层（除最后一个卷积层之外）之后，我们使用 SGD 对动量和批量归一化进行优化。我们分别使用 16 个图像的 mini-batches，将动量设置为 0.9，并使用 $5^{-4}$ 的权重衰减。学习率最初设定为 $10^{-4}$，当训练损失到极值时，学习率降至 $10^{-5}$。  

由于只有一小部分（1-2%）的网格单元包含文本，因此我们首先将非文本概率误差项与 0.01 相乘；随着训练的进行，这个权重逐渐增加到 1。由于类别不平衡，如果不使用这种加权方案，所有概率分数都会降至零。  

**推理** 我们从我们的 FCRN 模型中得到分类概率和边界框预测。通过对类别概率设置阈值对预测进行过滤。最后，使用非极大值抑制抑制来自邻近小区的多个检测，由此在两个重叠检测中抑制具有较低概率的检测。在下面，我们首先给出 t = 0.3 的保守阈值的结果，以获得更高的精度，然后将其放松到 t = 0.0（即接受所有提议）以获得更高的召回率。  

**评估协议** 我们使用文献中常用的两种协议报告文本检测性能 - （1）在 ICDAR 竞赛中广泛使用的评估定位方法的 DetEval，以及（2）PASCAL VOC 风格 intersection-over-union 重叠方法（≥0.5 IoU positive 检测）。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/5.jpg?raw=true)  

**单一和多尺度检测** 表 2 中的“FCRN 单尺度”条目显示了我们的 FCRN 模型在测试数据集上的表现。单尺度 FCRN 的最大 F 测量精度与 Neuman 等人的方法相当，而召回则明显恶化了12%。  

表2 中的“FCRN 多尺度”条目显示了我们网络多尺度应用的性能。这种方法比单尺度方法的最大召回提高了 12% 以上，并且胜过了 Neumann 等人的方法。    

**proposals 后处理** 当前的端到端文本识别（检测和识别）方法通过将检测与文本识别相结合来提高性能。为了进一步改进 FCRN 检测，我们使用FCRN 的多尺度检测作为提议并通过使用 Jaderberg 等人的后处理阶段对其进行改进。有三个阶段：首先使用二元 text/notext 随机森林分类器进行过滤；其次，使用 CNN 回归改进边界框；第三基于识别的 NMS，其中使用基于大型固定词典的 CNN 来识别单词图像，并且通过基于单词身份的非最大抑制来合并检测。详情见[20]。我们使用作者提供的代码进行公平比较。  

我们在两种模式下进行测试 - （1）low-recall：只使用高分（概率> 0.3）的多尺度 FCRN 检测（之前在单尺度和多尺度推理中使用的阈值）。这通常会产生少于 30 个提案。（2）high-recall：所有多尺度 FCRN 检测（通常约一千个）被使用。 这些方法在文本检测中的性能分别由表 2 中的名为“FCRN + multi-filt”和“FCRNall + multi-filt”的条目示出。请注意，low-recall 方法比现有技术更好，而 high-recall 方法则显着改善了所有数据集的 F-measure，改进 6% 的现状。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/6.jpg?raw=true)  

图 5 显示了 IC13 数据集上文本检测的 Precision-Recall 曲线。请注意，在使用多重过滤后处理进行修正之前，FCRN 多尺度检测输出的 high-recall（85.9%）。此外，与 Jaderberg 等人相比，注意“FCRNall + multi-filt”的最大召回率（+ 10.3%）和平均精确度（+ 11.1%）的急剧增加。  

此外，为了确定文本检测的改进是由于新的检测模型引起的，而不仅仅是由于我们的合成数据集的大尺寸，我们在 Wild 数据集的 SynthText 上训练了Jaderberg 等人的方法，图 5 和表 2 显示，即使有 10 倍以上（合成）的训练数据，Jaderberg 等人的模型也仅略有提高(AP +0.8%，recall +2.1%)。  

一种常见的失败模式是文本中不属于训练集的非常规字体。探测器也会被符号或图案的笔画宽度看起来像文本一样混淆，例如道路标志，棒图等。由于探测器不会缩放图像，所以不会检测到极小尺寸的文本实例。最后，由于字符之间的间距较大或较小，单词会分解为多个实例或合并为一个实例。  

### 4.3. Synthetic Dataset Evaluation  

我们调查合成文本场景数据生成流程的各个阶段对定位精度的贡献：我们生成三个合成训练数据集，其复杂程度越来越高，（1）放置在图像内的随机位置，（2）局限于局部颜色和纹理边界，以及（3）透视变形以匹配局部场景深度（同时也尊重上面（2）中的局部颜色和纹理边界）。数据集的所有其他方面保持不变-例如文字词典，背景图片，颜色分布。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/7.jpg?raw=true)  

图 6 显示了我们的方法“FCRNall + multi-filt”在 SVT 数据集上的定位结果。与随机放置相比，将文本限制在局部颜色和纹理区域可显着提高最大recall（+6.8％），AP（+3.85％）和 最大F-measure（+2.1％）。通过增加透视失真观察到边缘改善：AP为+0.75％，最大F-measure为+0.55％，最大 recall 没有变化。这可能是由于 SVT 数据集中的大多数文本实例处于前端并行方向。ICDAR 2013 数据集观察到类似的趋势，但更多的差异可能是由于ICDAR 的文本实例比 SVT 简单得多，并且从更高级的数据集中受益较少。  

### 4.4. End-to-End Text Spotting  

文本识别受检测阶段的限制，因为最先进的裁剪文字图像识别精度超过了98%。我们利用我们对文本定位的改进来获取文本识别中的最新结果。  

**评估协议** 除非另有说明，否则我们遵循 Wang 等人的标准评估协议。其中所有长度小于三个字符或包含非字母数字字符的单词都将被忽略。正检测需要至少 0.5 的重叠（IoU）。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/8.jpg?raw=true)  

表 3 显示了使用“FCRN + multi-filt”和“FCRNall + multi-filt”方法的端到端文本识别任务的结果。为了识别，我们使用基于 Jaderberg 等人的词汇编码 CNN 的管道中间记录识别阶段的输出。我们改进了之前报道的结果（F-measure）：ICDAR 数据集 +8%，SVT数据集 +3%。鉴于我们方法的高 recall（如图 5 所示），SVT 中许多文本实例未标记的事实导致精度下降；因此，我们看到 SVT 收益较小，SVT-50 收益较差。  

### 4.5. Timings  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/9.jpg?raw=true)  

在测试时，FCRN 可以在单一尺度下每秒处理 20 幅图像（大小为 512×512 像素），并且在 GPU 上以多个比例（1,1 / 2,1 / 4,1 / 8）运行时每秒处理大约 15 幅图像。当作为 Jaderberg 等人的文本定位流程中的高质量提案使用时。它取代了区域建议阶段，每个图像通常需要约 3 秒。因此，我们在区域提案阶段加快了约 45 倍。此外，“FCRN +多重过滤”方法仅使用多尺度 FCRN 的高分值检测，并在检测和端到端文本识别中实现了最新的结果，从而减少了项目后期阶段区域提议数量的 10 倍：Jaderberg 等人提议阶段出了大约 2000 个盒子，这些盒子使用随机森林分类器快速过滤到可过滤到约200个提议，而多尺度 FCRN 的高分数检测通常小于 30。表 4 比较了端到端文本识别所需的时间；我们的方法比 Jaderberg 等人的要快 3 倍到 23 倍，这取决于变体。    

## 5. Conclusion  

我们开发了一种新的 CNN 架构，用于在图像中生成文字区域提议。在可用的带注释的数据集上训练这种体系结构是不可能的，因为它们包含的样本太少，但我们已经表明，可以合成地生成足够逼真的训练图像，并且仅在这些图像上训练的 CNN 超过在真实图像上进行检测和端到端文本识别的最先进的性能。

## 参考  

- 1 [An End-to-End Trainable Neural Network for Image-based Sequence Recognition and Its Application to Scene Text Recognition](https://arxiv.org/abs/1507.05717) 
- 2 [CRNN论文翻译——中英文对照](https://blog.csdn.net/Quincuntial/article/details/77679419)  
- 3 [STN-OCR: A single Neural Network for Text Detection and Text Recognition](https://arxiv.org/abs/1707.08831)  
- 4 [Synthetic Data for Text Localisation in Natural Images](https://arxiv.org/abs/1604.06646)